
## PyZX Hybrid Mixed Quantum-Classical Circuit Demo
==========================================

This demo showcases PyZX's support for hybrid quantum-classical circuits using
the 'ground' feature. This allows modeling of measurements, classical control,
and mixed quantum-classical computation within the ZX-calculus framework.

The 'ground' feature in PyZX represents classical information flow and control,
enabling the representation of:
- Quantum measurements that produce classical bits
- Classical control of quantum gates (conditional operations)
- Mixed quantum-classical algorithms
- Teleportation and other protocols involving classical communication

In [ ]:
import pyzx as zx
from fractions import Fraction
import numpy as np
from typing import List, Tuple, Dict, Set

 In traditional ZX-calculus, graphs model quantum operations using Z and X spiders ```(nodes)``` connected by edges. This script extends that concept using "grounded" vertices, which represent classical information, such as measurement outcomes or classically controlled operations. By setting ```ground=True``` on certain vertices, PyZX can distinguish quantum behavior from classical control, enabling modeling of hybrid algorithms, such as quantum teleportation, conditional gates, and variational circuits.

Mathematically, grounding a vertex allows it to behave like a classical node in a computational graph, thus mixing quantum operations (unitary evolution) with non-unitary classical branching or feedback. The demo walks through six structured examples: (1) basic ground operations, (2) measurement circuits, (3) classically controlled gates, (4) quantum teleportation via classical bits, (5) how grounded vertices affect simplification, and (6) a hybrid variational algorithm combining classical parameter optimization with quantum circuits. Each example uses ZX-graph construction and PyZX's hybrid utilities like is_ground, ```grounds()```, and ```is_hybrid()``` to show how measurement, control, and classical logic are naturally incorporated into the quantum circuit graph structure. This demonstrates that PyZX's graphical model can support full hybrid quantum-classical computation rather than just unitary-only quantum circuits.

Let’s see how PyZX can be used to model hybrid quantum-classical circuits using its ground feature. This tutorial-style code walks through six core demonstrations using ```zx.Graph()``` objects—PyZX’s underlying graphical representation of quantum circuits, based on ZX-calculus. The key idea is that by marking certain vertices with ```ground=True```, we can distinguish classical information (like measurement outcomes or classical control signals) from purely quantum operations.

- Basic Ground Operations show how to create and inspect ground vertices.

- Measurement Circuits use ground vertices to represent quantum measurements (where quantum data collapses into classical bits).

- Conditional Gates demonstrate how classical bits can control quantum operations (e.g., applying an X gate based on a classical condition).

- Teleportation Protocol encodes a full quantum teleportation algorithm with classical communication channels modeled as ground vertices.

- Circuit Reduction highlights that ground-connected vertices affect how simplifications or adjoints of circuits are performed.

- Hybrid Algorithm Example combines classical parameters (from an optimizer) with a quantum circuit—mirroring how variational quantum algorithms work.

Mathematically, this models a hybrid computation graph: quantum operations remain unitary until measured, and classical control is handled via ground vertices that propagate classical bits through the graph. The PyZX API provides tools like is_ground, grounds, and is_hybrid to analyze and manage these mixed-mode systems. This hybrid extension enriches ZX-diagrams beyond pure quantum logic, enabling simulations of full hybrid algorithms such as variational circuits, feedback loops, and teleportation—all within one unified graph model.

In [37]:
class HybridCircuitDemo:
    """Demonstrates hybrid quantum-classical circuits using PyZX grounds."""

    def __init__(self):
        """Initialize the demo with basic setup."""
        print("=== PyZX Hybrid Quantum-Classical Circuit Demo ===\n")

    def basic_ground_operations(self):
        """Demonstrate basic ground vertex operations."""
        print("1. Basic Ground Operations")
        print("-" * 30)

        # Create a simple graph
        g = zx.Graph()

        # Add some vertices
        v1 = g.add_vertex(zx.VertexType.Z, qubit=0, row=1)
        v2 = g.add_vertex(zx.VertexType.X, qubit=1, row=1)
        v3 = g.add_vertex(zx.VertexType.Z, qubit=0, row=2)

        print(f"Created graph with {g.num_vertices()} vertices")

        # Set ground connections
        g.set_ground(v1, True)
        g.set_ground(v3, True)

        print(f"Vertex {v1} is ground-connected: {g.is_ground(v1)}")
        print(f"Vertex {v2} is ground-connected: {g.is_ground(v2)}")
        print(f"Vertex {v3} is ground-connected: {g.is_ground(v3)}")

        # Get all ground vertices
        ground_vertices = g.grounds()
        print(f"Ground vertices: {ground_vertices}")

        # Check if graph is hybrid
        print(f"Graph is hybrid (has grounds): {g.is_hybrid()}")

        print()

    def measurement_circuit(self):
        """Demonstrate a circuit with quantum measurements."""
        print("2. Quantum Measurement Circuit")
        print("-" * 30)

        # Create a circuit that prepares a Bell state and measures both qubits
        g = zx.Graph()

        # Add input/output boundaries
        input1 = g.add_vertex(zx.VertexType.BOUNDARY, qubit=0, row=0)
        input2 = g.add_vertex(zx.VertexType.BOUNDARY, qubit=1, row=0)

        # Hadamard on first qubit (represented as Z-spider with phase π/2)
        h_gate = g.add_vertex(zx.VertexType.Z, qubit=0, row=1, phase=Fraction(1,2))

        # CNOT gate (represented as connected Z and X spiders)
        cnot_control = g.add_vertex(zx.VertexType.Z, qubit=0, row=2)
        cnot_target = g.add_vertex(zx.VertexType.X, qubit=1, row=2)

        # Measurement vertices (ground-connected to represent classical output)
        measure1 = g.add_vertex(zx.VertexType.Z, qubit=0, row=3, ground=True)
        measure2 = g.add_vertex(zx.VertexType.Z, qubit=1, row=3, ground=True)

        # Classical outputs
        output1 = g.add_vertex(zx.VertexType.BOUNDARY, qubit=0, row=4)
        output2 = g.add_vertex(zx.VertexType.BOUNDARY, qubit=1, row=4)

        # Connect the circuit
        g.add_edge((input1, h_gate))
        g.add_edge((input2, cnot_target))
        g.add_edge((h_gate, cnot_control))
        g.add_edge((cnot_control, cnot_target))  # CNOT connection
        g.add_edge((cnot_control, measure1))
        g.add_edge((cnot_target, measure2))
        g.add_edge((measure1, output1))
        g.add_edge((measure2, output2))

        # Set inputs/outputs
        g.set_inputs((input1, input2))
        g.set_outputs((output1, output2))

        print(f"Bell state measurement circuit created")
        print(f"Number of vertices: {g.num_vertices()}")
        print(f"Number of ground vertices: {len(g.grounds())}")
        print(f"Ground vertices: {g.grounds()}")
        print(f"Circuit is hybrid: {g.is_hybrid()}")

        # Analyze the measurement structure
        print("\nMeasurement Analysis:")
        for v in g.grounds():
            neighbors = g.neighbors(v)
            print(f"  Ground vertex {v}: connected to {neighbors}")
            print(f"  Type: {g.type(v)}, Phase: {g.phase(v)}")

        print()

    def conditional_gate_circuit(self):
        """Demonstrate classical control of quantum gates."""
        print("3. Classical Control Circuit")
        print("-" * 30)

        g = zx.Graph()

        # Create a circuit where a quantum gate is controlled by a classical bit
        # This models: if (classical_bit) then apply_X_gate()

        # Quantum input
        q_input = g.add_vertex(zx.VertexType.BOUNDARY, qubit=0, row=0)

        # Classical control input (ground-connected)
        c_input = g.add_vertex(zx.VertexType.BOUNDARY, qubit=1, row=0, ground=True)

        # Control logic vertex (processes classical information)
        control_logic = g.add_vertex(zx.VertexType.Z, qubit=1, row=1, ground=True)

        # Controlled quantum gate (X gate, controlled by classical bit)
        controlled_x = g.add_vertex(zx.VertexType.X, qubit=0, row=2, phase=Fraction(1))

        # Control connection vertex (mediates classical-quantum interaction)
        control_conn = g.add_vertex(zx.VertexType.Z, qubit=0, row=1, ground=True)

        # Quantum output
        q_output = g.add_vertex(zx.VertexType.BOUNDARY, qubit=0, row=3)

        # Classical output (copy of control bit)
        c_output = g.add_vertex(zx.VertexType.BOUNDARY, qubit=1, row=3, ground=True)

        # Connect the circuit
        g.add_edge((q_input, control_conn))
        g.add_edge((c_input, control_logic))
        g.add_edge((control_logic, control_conn))  # Classical control
        g.add_edge((control_conn, controlled_x))
        g.add_edge((controlled_x, q_output))
        g.add_edge((control_logic, c_output))  # Classical output

        g.set_inputs((q_input, c_input))
        g.set_outputs((q_output, c_output))

        print(f"Conditional gate circuit created")
        print(f"Hybrid circuit: {g.is_hybrid()}")
        print(f"Ground vertices: {len(g.grounds())}")

        # Analyze classical control structure
        print("\nClassical Control Analysis:")
        classical_vertices = g.grounds()
        for v in classical_vertices:
            neighbors = [n for n in g.neighbors(v)]
            quantum_neighbors = [n for n in neighbors if not g.is_ground(n)]
            classical_neighbors = [n for n in neighbors if g.is_ground(n)]

            print(f"  Classical vertex {v}:")
            print(f"    Quantum connections: {quantum_neighbors}")
            print(f"    Classical connections: {classical_neighbors}")

        print()

    def teleportation_protocol(self):
        """Demonstrate quantum teleportation with classical communication."""
        print("4. Quantum Teleportation Protocol")
        print("-" * 30)

        g = zx.Graph()

        # Teleportation involves:
        # 1. Unknown quantum state to be teleported
        # 2. Entangled Bell pair shared between Alice and Bob
        # 3. Alice's Bell measurement (produces 2 classical bits)
        # 4. Bob's conditional operations based on classical bits

        # Alice's unknown state input
        alice_input = g.add_vertex(zx.VertexType.BOUNDARY, qubit=0, row=0)

        # Bell pair inputs (|00⟩ + |11⟩)/√2
        bell_alice = g.add_vertex(zx.VertexType.BOUNDARY, qubit=1, row=0)
        bell_bob = g.add_vertex(zx.VertexType.BOUNDARY, qubit=2, row=0)

        # Bell pair preparation
        h_bell = g.add_vertex(zx.VertexType.Z, qubit=1, row=1, phase=Fraction(1,2))
        cnot_bell_ctrl = g.add_vertex(zx.VertexType.Z, qubit=1, row=2)
        cnot_bell_targ = g.add_vertex(zx.VertexType.X, qubit=2, row=2)

        # Alice's Bell measurement
        alice_cnot_ctrl = g.add_vertex(zx.VertexType.Z, qubit=0, row=3)
        alice_cnot_targ = g.add_vertex(zx.VertexType.X, qubit=1, row=3)
        alice_h = g.add_vertex(zx.VertexType.Z, qubit=0, row=4, phase=Fraction(1,2))

        # Alice's measurement outcomes (classical bits)
        measure_x = g.add_vertex(zx.VertexType.Z, qubit=0, row=5, ground=True)
        measure_z = g.add_vertex(zx.VertexType.Z, qubit=1, row=5, ground=True)

        # Classical communication to Bob
        comm_x = g.add_vertex(zx.VertexType.Z, qubit=0, row=6, ground=True)
        comm_z = g.add_vertex(zx.VertexType.Z, qubit=1, row=6, ground=True)

        # Bob's conditional operations
        bob_x_gate = g.add_vertex(zx.VertexType.X, qubit=2, row=7, phase=Fraction(1))
        bob_z_gate = g.add_vertex(zx.VertexType.Z, qubit=2, row=8, phase=Fraction(1))

        # Bob's output (reconstructed state)
        bob_output = g.add_vertex(zx.VertexType.BOUNDARY, qubit=2, row=9)

        # Connect Bell pair preparation
        g.add_edge((bell_alice, h_bell))
        g.add_edge((bell_bob, cnot_bell_targ))
        g.add_edge((h_bell, cnot_bell_ctrl))
        g.add_edge((cnot_bell_ctrl, cnot_bell_targ))

        # Connect Alice's operations
        g.add_edge((alice_input, alice_cnot_ctrl))
        g.add_edge((cnot_bell_ctrl, alice_cnot_targ))
        g.add_edge((alice_cnot_ctrl, alice_cnot_targ))
        g.add_edge((alice_cnot_ctrl, alice_h))
        g.add_edge((alice_h, measure_x))
        g.add_edge((alice_cnot_targ, measure_z))

        # Classical communication
        g.add_edge((measure_x, comm_x))
        g.add_edge((measure_z, comm_z))

        # Bob's conditional operations (classical control)
        g.add_edge((cnot_bell_targ, bob_x_gate))
        g.add_edge((comm_x, bob_x_gate))  # Classical control
        g.add_edge((bob_x_gate, bob_z_gate))
        g.add_edge((comm_z, bob_z_gate))  # Classical control
        g.add_edge((bob_z_gate, bob_output))

        g.set_inputs((alice_input, bell_alice, bell_bob))
        g.set_outputs((bob_output,))

        print(f"Teleportation protocol circuit created")
        print(f"Total vertices: {g.num_vertices()}")
        print(f"Ground (classical) vertices: {len(g.grounds())}")
        print(f"Classical communication channels: {len([v for v in g.grounds() if 'comm' in str(v)])}")

        # Analyze information flow
        print("\nInformation Flow Analysis:")
        print("Classical vertices and their roles:")
        ground_vertices = list(g.grounds())
        for i, v in enumerate(ground_vertices):
            neighbors = g.neighbors(v)
            if any('measure' in str(n) for n in neighbors):
                print(f"  Vertex {v}: Measurement outcome")
            elif any('comm' in str(n) for n in neighbors):
                print(f"  Vertex {v}: Classical communication")
            else:
                print(f"  Vertex {v}: Classical control")

        print()

    def reduction_with_grounds(self):
        """Demonstrate how ground vertices affect circuit reduction."""
        print("5. Circuit Reduction with Grounds")
        print("-" * 30)

        # Create a simple circuit with some redundancy
        g = zx.Graph()

        input1 = g.add_vertex(zx.VertexType.BOUNDARY, qubit=0, row=0)

        # Identity-like structure that should be reducible
        z1 = g.add_vertex(zx.VertexType.Z, qubit=0, row=1, phase=0)
        z2 = g.add_vertex(zx.VertexType.Z, qubit=0, row=2, phase=0)

        # But one vertex is ground-connected (measurement)
        measure = g.add_vertex(zx.VertexType.Z, qubit=0, row=3, ground=True)

        output1 = g.add_vertex(zx.VertexType.BOUNDARY, qubit=0, row=4)

        g.add_edge((input1, z1))
        g.add_edge((z1, z2))
        g.add_edge((z2, measure))
        g.add_edge((measure, output1))

        g.set_inputs((input1,))
        g.set_outputs((output1,))

        print("Original circuit:")
        print(f"  Vertices: {g.num_vertices()}")
        print(f"  Edges: {g.num_edges()}")
        print(f"  Ground vertices: {len(g.grounds())}")
        print(f"  Is hybrid: {g.is_hybrid()}")

        # Show that grounds are preserved during operations
        g_copy = g.copy()
        print(f"\nAfter copying:")
        print(f"  Ground vertices preserved: {len(g_copy.grounds()) == len(g.grounds())}")
        print(f"  Ground vertices: {g_copy.grounds()}")

        # Demonstrate adjoint with grounds
        g_adj = g.adjoint()
        print(f"\nAfter taking adjoint:")
        print(f"  Ground vertices: {len(g_adj.grounds())}")
        print(f"  Still hybrid: {g_adj.is_hybrid()}")

        print()

    def hybrid_algorithm_example(self):
        """Show a complete hybrid quantum-classical algorithm."""
        print("6. Hybrid Algorithm Example: Quantum-Classical Feedback")
        print("-" * 50)

        # Simulate a variational quantum algorithm with classical optimization
        g = zx.Graph()

        # Quantum register
        q_inputs = []
        for i in range(3):
            q_inputs.append(g.add_vertex(zx.VertexType.BOUNDARY, qubit=i, row=0))

        # Classical parameter inputs (from optimizer)
        c_params = []
        for i in range(2):
            c_params.append(g.add_vertex(zx.VertexType.BOUNDARY, qubit=3+i, row=0, ground=True))

        # Parameterized quantum circuit
        # Layer 1: RY gates controlled by classical parameters
        ry_gates = []
        for i in range(3):
            ry = g.add_vertex(zx.VertexType.Z, qubit=i, row=1, phase=Fraction(1,4))
            param_ctrl = g.add_vertex(zx.VertexType.Z, qubit=3, row=1, ground=True)
            ry_gates.append(ry)
            g.add_edge((q_inputs[i], ry))
            g.add_edge((c_params[0], param_ctrl))
            g.add_edge((param_ctrl, ry))  # Classical control of gate parameter

        # Layer 2: Entangling gates
        entangling = []
        for i in range(2):
            cnot_ctrl = g.add_vertex(zx.VertexType.Z, qubit=i, row=2)
            cnot_targ = g.add_vertex(zx.VertexType.X, qubit=i+1, row=2)
            entangling.extend([cnot_ctrl, cnot_targ])
            g.add_edge((ry_gates[i], cnot_ctrl))
            g.add_edge((ry_gates[i+1], cnot_targ))
            g.add_edge((cnot_ctrl, cnot_targ))

        # Measurements for expectation value estimation
        measurements = []
        for i in range(3):
            measure = g.add_vertex(zx.VertexType.Z, qubit=i, row=3, ground=True)
            measurements.append(measure)
            if i < 2:
                g.add_edge((entangling[2*i], measure))
            else:
                g.add_edge((ry_gates[i], measure))

        # Classical processing of measurement results
        classical_proc = g.add_vertex(zx.VertexType.Z, qubit=5, row=4, ground=True, phase=0)
        for m in measurements:
            g.add_edge((m, classical_proc))

        # Classical outputs (cost function value, updated parameters)
        cost_output = g.add_vertex(zx.VertexType.BOUNDARY, qubit=5, row=5, ground=True)
        param_output = g.add_vertex(zx.VertexType.BOUNDARY, qubit=6, row=5, ground=True)

        g.add_edge((classical_proc, cost_output))
        g.add_edge((c_params[1], param_output))

        g.set_inputs(tuple(q_inputs + c_params))
        g.set_outputs((cost_output, param_output))

        print(f"Hybrid variational algorithm circuit:")
        print(f"  Total vertices: {g.num_vertices()}")
        print(f"  Quantum vertices: {g.num_vertices() - len(g.grounds())}")
        print(f"  Classical vertices: {len(g.grounds())}")
        print(f"  Classical inputs: {len([v for v in c_params])}")
        print(f"  Classical outputs: 2")

        # Analyze the hybrid structure
        print(f"\nHybrid Structure Analysis:")
        print(f"  Classical parameter control: {len([v for v in g.grounds() if g.row(v) == 1])}")
        print(f"  Quantum measurements: {len(measurements)}")
        print(f"  Classical processing nodes: 1")
        print(f"  Feedback loops: Classical params → Quantum gates → Measurements → Classical processing")

        print()

    def run_all_demos(self):
        """Run all demonstration functions."""
        self.basic_ground_operations()
        self.measurement_circuit()
        self.conditional_gate_circuit()
        self.teleportation_protocol()
        self.reduction_with_grounds()
        self.hybrid_algorithm_example()

        print("=== Demo Summary ===")
        print()
        print("This demo has shown how PyZX's 'ground' feature enables:")
        print("• Representation of quantum measurements as classical outputs")
        print("• Classical control of quantum gates and operations")
        print("• Hybrid quantum-classical algorithms and protocols")
        print("• Quantum teleportation with classical communication")
        print("• Variational quantum algorithms with classical optimization")
        print()
        print("Key concepts:")
        print("• ground=True: Marks vertices as carrying classical information")
        print("• is_ground(v): Check if vertex is ground-connected")
        print("• grounds(): Get all ground vertices in the graph")
        print("• is_hybrid(): Check if graph contains classical components")
        print("• Ground vertices are preserved during graph operations (copy, adjoint)")
        print()
        print("The ground feature extends PyZX beyond pure quantum circuits to")
        print("full hybrid quantum-classical computation models!")

def main():
    """Main function to run the demo."""
    try:
        demo = HybridCircuitDemo()
        demo.run_all_demos()
    except ImportError as e:
        print(f"Error: PyZX not found. Please install PyZX first: pip install pyzx")
        print(f"ImportError details: {e}")
    except Exception as e:
        print(f"Error running demo: {e}")
        import traceback
        traceback.print_exc()

if __name__ == "__main__":
    main()


=== PyZX Hybrid Quantum-Classical Circuit Demo ===

1. Basic Ground Operations
------------------------------
Created graph with 3 vertices
Vertex 0 is ground-connected: True
Vertex 1 is ground-connected: False
Vertex 2 is ground-connected: True
Ground vertices: {0, 2}
Graph is hybrid (has grounds): True

2. Quantum Measurement Circuit
------------------------------
Bell state measurement circuit created
Number of vertices: 9
Number of ground vertices: 2
Ground vertices: {5, 6}
Circuit is hybrid: True

Measurement Analysis:
  Ground vertex 5: connected to dict_keys([3, 7])
  Type: 1, Phase: 0
  Ground vertex 6: connected to dict_keys([4, 8])
  Type: 1, Phase: 0

3. Classical Control Circuit
------------------------------
Conditional gate circuit created
Hybrid circuit: True
Ground vertices: 4

Classical Control Analysis:
  Classical vertex 1:
    Quantum connections: []
    Classical connections: [2]
  Classical vertex 2:
    Quantum connections: []
    Classical connections: [1, 4, 6]

Let’s see how to build a simple classical circuit using the ZX-calculus with the `pyzx` library. In this introductory example, we construct a linear ZX-diagram that represents a basic classical signal flow using *grounded vertices*, which indicate classical (non-quantum) behavior. We start by creating an empty `Graph` object. Then, we add a **boundary input vertex** (`in_v`) at qubit 0 and row 0, with the `ground=True` flag to indicate it's classical. Next, we place two **Z-spiders** (`z1` and `z2`) at subsequent rows, also marked as classical. Finally, we add a **boundary output vertex** (`out_v`) at row 3. These vertices are connected sequentially using edges, forming a simple chain: input → Z → Z → output. We define the first and last vertices as the inputs and outputs of the graph, respectively. Mathematically, this structure resembles a classical signal passing through two identity operations (Z-spiders without phase), with no quantum entanglement or phase manipulation involved. It forms the simplest backbone for understanding classical data flow in ZX-diagrams.


In [ ]:
import pyzx as zx
import timeit
from fractions import Fraction
from typing import List


def build_simple_classical_circuit() -> zx.Graph:
    """Build a basic classical-only circuit using ground vertices."""
    g = zx.Graph()
    in_v = g.add_vertex(zx.VertexType.BOUNDARY, qubit=0, row=0, ground=True)
    z1 = g.add_vertex(zx.VertexType.Z, qubit=0, row=1, ground=True)
    z2 = g.add_vertex(zx.VertexType.Z, qubit=0, row=2, ground=True)
    out_v = g.add_vertex(zx.VertexType.BOUNDARY, qubit=0, row=3, ground=True)

    g.add_edge((in_v, z1))
    g.add_edge((z1, z2))
    g.add_edge((z2, out_v))

    g.set_inputs((in_v,))
    g.set_outputs((out_v,))
    return g

Let’s see how to construct both a simple quantum and a hybrid quantum-classical circuit using the pyzx library and ZX-diagrams. These examples help us understand how to model quantum gates and classical control within the same graphical framework.

Simple Quantum Circuit
In build_simple_quantum_circuit, we start with a single-qubit Circuit object and sequentially apply three quantum gates:

- A Hadamard (H) gate, which creates superposition.

- A T gate, implemented via ZPhase with angle π/4 (represented as Fraction(1, 8) of a full turn).

- A Pauli-X gate, encoded as XPhase with phase 1 (a full π rotation).
This circuit is then converted into a ZX-diagram using .to_graph(). Mathematically, this forms a purely quantum linear circuit with phase information, with no classical influence.

Hybrid Quantum-Classical Circuit
In build_simple_hybrid_circuit, we model interaction between quantum and classical information. We define:

- A quantum input (q_in) on qubit 0 and a classical control input (c_ctrl) on qubit 1 (indicated by ground=True).

- An X-spider (xgate) representing a Pauli-X operation on the quantum wire.

- A Z-spider (classical_condition) that is grounded, indicating classical control.

We connect:

- q_in to the xgate (quantum flow),

- c_ctrl to the classical_condition, and

- classical_condition to the xgate, forming a control structure where the classical node influences the quantum gate.

This is completed by adding a quantum output (q_out) after the gate. Mathematically, this diagram reflects a conditional operation where the quantum state evolves based on classical input, demonstrating how ZX-diagrams naturally express hybrid control logic between classical and quantum domains.

In [ ]:
def build_simple_quantum_circuit() -> zx.Graph:
    """Build a basic quantum-only circuit without ground nodes."""
    circ = zx.Circuit(1)
    circ.add_gate("H", 0)
    circ.add_gate("ZPhase", 0, Fraction(1, 8))  # T gate
    circ.add_gate("XPhase", 0, Fraction(1))     # Pauli X
    return circ.to_graph()


def build_simple_hybrid_circuit() -> zx.Graph:
    """Build a basic hybrid quantum-classical circuit using ground."""
    g = zx.Graph()
    q_in = g.add_vertex(zx.VertexType.BOUNDARY, qubit=0, row=0)
    c_ctrl = g.add_vertex(zx.VertexType.BOUNDARY, qubit=1, row=0, ground=True)

    xgate = g.add_vertex(zx.VertexType.X, qubit=0, row=1, phase=Fraction(1))
    classical_condition = g.add_vertex(zx.VertexType.Z, qubit=0, row=0, ground=True)

    g.add_edge((q_in, xgate))
    g.add_edge((c_ctrl, classical_condition))
    g.add_edge((classical_condition, xgate))

    q_out = g.add_vertex(zx.VertexType.BOUNDARY, qubit=0, row=2)
    g.add_edge((xgate, q_out))

    g.set_inputs((q_in, c_ctrl))
    g.set_outputs((q_out,))
    return g

Let’s see how we can benchmark and compare different types of circuits—classical-only, quantum-only, and hybrid quantum-classical—using the pyzx library. The goal here is to analyze both the structure and simplification performance of each circuit type using ZX-diagram representations.

Benchmarking Function
The benchmark_circuit function takes in:

A graph_builder function (like build_simple_quantum_circuit),

A human-readable name,

And the number of trials (default: 100).

It measures the average time (in milliseconds) required to build and fully simplify the ZX-graph using zx.simplify.full_reduce. It also reports structural properties:

Number of vertices and edges in the graph,

Number of grounded nodes (used for classical inputs),

Whether the circuit is hybrid (contains both quantum and classical components).

Main Routine
The main() function orchestrates the benchmarking:

It defines three types of circuits—Classical, Quantum, and Hybrid—each constructed by its respective builder function.

For each, it runs the benchmark and collects performance data.

Finally, it prints a summary comparing:

Circuit size (vertices, edges),

Use of classical control (grounds, hybrid status),

And average simplification time, which reflects computational overhead.

In [28]:
def benchmark_circuit(graph_builder, name: str, num_trials: int = 100) -> dict:
    """Benchmark a graph build function."""
    def run_once():
        g = graph_builder()
        zx.simplify.full_reduce(g)

    time_taken = timeit.timeit(run_once, number=num_trials)
    g = graph_builder()
    return {
        "name": name,
        "vertices": g.num_vertices(),
        "edges": g.num_edges(),
        "grounds": len(g.grounds()) if g.is_hybrid() else 0,
        "hybrid": g.is_hybrid(),
        "avg_time_ms": round((time_taken / num_trials) * 1000, 4)
    }


def main():
    print("=== Hybrid Quantum-Classical Circuit Benchmark ===\n")

    circuits = [
        ("Classical Only", build_simple_classical_circuit),
        ("Quantum Only", build_simple_quantum_circuit),
        ("Hybrid Circuit", build_simple_hybrid_circuit),
    ]

    results = []

    for name, builder in circuits:
        print(f"Benchmarking: {name}")
        stats = benchmark_circuit(builder, name)
        results.append(stats)

    print("\n--- Summary ---")
    for res in results:
        print(
            f"{res['name']} → Vertices: {res['vertices']}, Edges: {res['edges']}, "
            f"Grounds: {res['grounds']}, Hybrid: {res['hybrid']}, "
            f"Avg Time: {res['avg_time_ms']} ms"
        )


if __name__ == "__main__":
    main()


=== Hybrid Quantum-Classical Circuit Benchmark ===

Benchmarking: Classical Only
Benchmarking: Quantum Only
Benchmarking: Hybrid Circuit

--- Summary ---
Classical Only → Vertices: 4, Edges: 3, Grounds: 4, Hybrid: True, Avg Time: 0.1908 ms
Quantum Only → Vertices: 5, Edges: 4, Grounds: 0, Hybrid: False, Avg Time: 0.2383 ms
Hybrid Circuit → Vertices: 5, Edges: 4, Grounds: 2, Hybrid: True, Avg Time: 0.1709 ms



## PyZX Hybrid Quantum-Classical Circuit Extension
================================================

This extended demo explores additional hybrid quantum-classical circuit functionality
using the 'ground' feature in PyZX. The original framework is preserved while adding
new circuits for classical fanout, hybrid multiplexing, and measurement-controlled routing.
"""

Let’s see how to build more advanced hybrid quantum-classical circuits using PyZX. This extended example demonstrates three hybrid patterns: classical fanout, hybrid multiplexing, and measurement-controlled routing—each showing different interactions between classical control and quantum data. These circuits use ground nodes to represent classical (non-coherent) values alongside quantum wires in ZX-diagrams.

## 1. Classical Fanout Circuit
This demonstrates copying a classical bit (from a single classical input) to multiple classical outputs—a non-quantum operation allowed in classical logic but forbidden in quantum circuits (due to the no-cloning theorem).

A classical input is created with ground=True.

Two Z-spiders (also grounded) act as fan-out junctions.

Two classical outputs receive the copied bits.

Ground nodes: All except the boundary are marked classical.

ℹ️ Uses g.grounds() to count and verify grounded vertices. Useful to distinguish classical logic within hybrid systems.

## 2. Hybrid Multiplexer Circuit
Here, a classical selector bit determines which of two quantum inputs is routed forward—a simple multiplexing operation based on classical control.

A classical selector node is marked with ground=True.

Two quantum inputs enter on different wires.

A classical control Z node influences two quantum gates: mux_z and mux_x (Z and X spiders).

Both gate outputs merge at a single quantum output.

💡 Demonstrates hybrid control flow—classical bits conditionally triggering operations on quantum wires.

## 3. Measurement-Controlled Routing
This shows a quantum measurement creating a classical control, which is then used to route a quantum state to different output ports.

A quantum input goes into a measuring Z-spider (treated as classical via ground=True).

Its outcome drives a classical control Z spider.

The input is partially preserved to a qmid node (representing unmeasured path).

Depending on the control value, either out1 or out2 is selected.

🧠 This simulates a mid-circuit measurement, where the measurement outcome affects downstream circuit behavior—a common feature in NISQ-era hybrid algorithms.

In [60]:
import pyzx as zx
from fractions import Fraction
from typing import List, Tuple, Dict, Set

class HybridCircuitExtension:
    def __init__(self):
        print("=== Extended PyZX Hybrid Quantum-Classical Circuit Demo ===\n")

    def classical_fanout(self):
        """Demonstrate classical fan-out: copying a classical bit to multiple locations."""
        print("7. Classical Fanout Circuit")
        print("-" * 30)

        g = zx.Graph()

        # Classical input bit
        classical_input = g.add_vertex(zx.VertexType.BOUNDARY, qubit=0, row=0, ground=True)

        # Fan-out nodes (e.g., copy to 2 locations)
        fan1 = g.add_vertex(zx.VertexType.Z, qubit=0, row=1, ground=True)
        fan2 = g.add_vertex(zx.VertexType.Z, qubit=0, row=1, ground=True)

        # Classical outputs
        out1 = g.add_vertex(zx.VertexType.BOUNDARY, qubit=1, row=2, ground=True)
        out2 = g.add_vertex(zx.VertexType.BOUNDARY, qubit=2, row=2, ground=True)

        g.add_edge((classical_input, fan1))
        g.add_edge((classical_input, fan2))
        g.add_edge((fan1, out1))
        g.add_edge((fan2, out2))

        g.set_inputs((classical_input,))
        g.set_outputs((out1, out2))

        print(f"Fanout circuit created with {g.num_vertices()} vertices")
        print(f"Ground (classical) vertices: {len(g.grounds())}")
        print(f"Graph is hybrid: {g.is_hybrid()}\n")

    def hybrid_multiplexer(self):
        """Demonstrate a hybrid multiplexer circuit: select quantum data line via classical control."""
        print("8. Hybrid Multiplexer Circuit")
        print("-" * 30)

        g = zx.Graph()

        # Classical selector (ground)
        sel = g.add_vertex(zx.VertexType.BOUNDARY, qubit=0, row=0, ground=True)

        # Quantum inputs
        q1 = g.add_vertex(zx.VertexType.BOUNDARY, qubit=1, row=0)
        q2 = g.add_vertex(zx.VertexType.BOUNDARY, qubit=2, row=0)

        # Control logic to switch inputs
        ctrl = g.add_vertex(zx.VertexType.Z, qubit=0, row=1, ground=True)

        # Multiplexer logic: apply Z or X gate based on selector
        mux_z = g.add_vertex(zx.VertexType.Z, qubit=1, row=2)
        mux_x = g.add_vertex(zx.VertexType.X, qubit=2, row=2)

        # Quantum output
        out = g.add_vertex(zx.VertexType.BOUNDARY, qubit=3, row=3)

        g.add_edge((sel, ctrl))
        g.add_edge((ctrl, mux_z))
        g.add_edge((ctrl, mux_x))
        g.add_edge((q1, mux_z))
        g.add_edge((q2, mux_x))
        g.add_edge((mux_z, out))
        g.add_edge((mux_x, out))

        g.set_inputs((sel, q1, q2))
        g.set_outputs((out,))

        print(f"Multiplexer circuit created with {g.num_vertices()} vertices")
        print(f"Hybrid: {g.is_hybrid()}")
        print(f"Ground vertices: {g.grounds()}\n")

    def measurement_controlled_routing(self):
        """Use measurement result to route a quantum state to different output paths."""
        print("9. Measurement-Controlled Routing")
        print("-" * 30)

        g = zx.Graph()

        # Quantum input
        qin = g.add_vertex(zx.VertexType.BOUNDARY, qubit=0, row=0)

        # Mid-circuit measurement (creates classical control bit)
        measure = g.add_vertex(zx.VertexType.Z, qubit=0, row=1, ground=True)

        # Classical control node
        ctrl = g.add_vertex(zx.VertexType.Z, qubit=1, row=2, ground=True)

        # Routing logic: if ctrl==0 -> out1; if ctrl==1 -> out2
        qmid = g.add_vertex(zx.VertexType.Z, qubit=0, row=3)
        out1 = g.add_vertex(zx.VertexType.BOUNDARY, qubit=1, row=4)
        out2 = g.add_vertex(zx.VertexType.BOUNDARY, qubit=2, row=4)

        g.add_edge((qin, measure))
        g.add_edge((measure, ctrl))
        g.add_edge((measure, qmid))
        g.add_edge((ctrl, out1))
        g.add_edge((qmid, out2))

        g.set_inputs((qin,))
        g.set_outputs((out1, out2))

        print(f"Measurement routing circuit created")
        print(f"Hybrid: {g.is_hybrid()} | Ground vertices: {g.grounds()}\n")


if __name__ == "__main__":
    ext = HybridCircuitExtension()
    ext.classical_fanout()
    ext.hybrid_multiplexer()
    ext.measurement_controlled_routing()


=== Extended PyZX Hybrid Quantum-Classical Circuit Demo ===

7. Classical Fanout Circuit
------------------------------
Fanout circuit created with 5 vertices
Ground (classical) vertices: 5
Graph is hybrid: True

8. Hybrid Multiplexer Circuit
------------------------------
Multiplexer circuit created with 7 vertices
Hybrid: True
Ground vertices: {0, 3}

9. Measurement-Controlled Routing
------------------------------
Measurement routing circuit created
Hybrid: True | Ground vertices: {1, 2}




## Extended PyZX Hybrid Quantum-Classical Circuit Demo
==================================================

This extension explores additional functionality of hybrid circuits
including feedback control, classical bit fanout, dynamic reconfiguration,
and mixed measurement-feedback loops in the PyZX ZX-calculus framework.


Let’s explore more advanced hybrid quantum-classical circuit structures in PyZX, highlighting how classical information (from measurements or inputs) can dynamically affect quantum processing. The following examples extend hybrid circuit design into feedback control, fanout with multi-qubit gates, and cascading adaptive behavior—important for realistic quantum-classical algorithms.

## 1. Classical Feedback Loop
This circuit models a feedback-controlled quantum operation, where a measured quantum state informs a future operation on the same qubit.

A quantum input is measured early (meas) using a grounded Z-spider.

The measurement outcome is passed to a classical processor (classical_proc, also grounded).

A conditional quantum gate (feedback_gate) is applied using that classical value.

A final measurement captures the result.

🔁 The key loop: quantum → classical (meas) → feedback gate → quantum.
✅ Grounded nodes (ground=True) represent classical data from measurement and logic.

## 2. Classical Fan-Out with Conditional Gates
Here, a single classical input controls two different quantum operations, mimicking parallel classical control logic.

A classical control bit is introduced via a boundary vertex with ground=True.

That control is fanned out to influence:

A Pauli-X gate on q1

A Pauli-Z gate on q2

Quantum paths are independent, but classically synchronized.

📌 Highlights multi-target classical control, showing that classical signals can coordinate actions across different quantum channels in hybrid circuits.

## 3. Adaptive Measurement Cascade
This circuit builds a cascading control flow: each quantum operation is conditionally executed based on successive measurements.

The quantum input flows through:

A first measurement → classical control → conditional gate

Then another measurement → classical control → second conditional gate

Each measurement result affects the next quantum operation.

📈 This reflects dynamic/adaptive behavior, like real-time decision-making or branching logic, based on measurement outcomes.


## Extended PyZX Hybrid Quantum-Classical Circuit Demo
==================================================

This extension explores additional functionality of hybrid circuits
including feedback control, classical bit fanout, dynamic reconfiguration,
and mixed measurement-feedback loops in the PyZX ZX-calculus framework.


In [62]:
import pyzx as zx
from fractions import Fraction

class ExtendedHybridCircuitDemo:
    def __init__(self):
        print("=== Extended PyZX Hybrid Quantum-Classical Circuit Demo ===\n")

    def classical_feedback_loop(self):
        """Demonstrates classical feedback control with a loop."""
        print("7. Classical Feedback Loop")
        print("-" * 30)

        g = zx.Graph()

        # Quantum input
        q_in = g.add_vertex(zx.VertexType.BOUNDARY, qubit=0, row=0)

        # Initial measurement
        meas = g.add_vertex(zx.VertexType.Z, qubit=0, row=1, ground=True)

        # Classical processing and feedback gate
        classical_proc = g.add_vertex(zx.VertexType.Z, qubit=1, row=2, ground=True)
        feedback_gate = g.add_vertex(zx.VertexType.X, qubit=0, row=3, phase=Fraction(1))

        # Final measurement
        final_meas = g.add_vertex(zx.VertexType.Z, qubit=0, row=4, ground=True)
        q_out = g.add_vertex(zx.VertexType.BOUNDARY, qubit=0, row=5)

        # Connect components
        g.add_edge((q_in, meas))
        g.add_edge((meas, classical_proc))
        g.add_edge((classical_proc, feedback_gate))
        g.add_edge((feedback_gate, final_meas))
        g.add_edge((final_meas, q_out))

        g.set_inputs((q_in,))
        g.set_outputs((q_out,))

        print(f"Vertices: {g.num_vertices()}, Grounds: {len(g.grounds())}, Is hybrid: {g.is_hybrid()}\n")

    def classical_fanout_and_condition(self):
        """Demonstrates classical fan-out control over multiple qubits."""
        print("8. Classical Fan-Out and Conditional Gates")
        print("-" * 30)

        g = zx.Graph()

        # Classical control input
        c_in = g.add_vertex(zx.VertexType.BOUNDARY, qubit=2, row=0, ground=True)
        control = g.add_vertex(zx.VertexType.Z, qubit=2, row=1, ground=True)

        # Two quantum inputs
        q1 = g.add_vertex(zx.VertexType.BOUNDARY, qubit=0, row=0)
        q2 = g.add_vertex(zx.VertexType.BOUNDARY, qubit=1, row=0)

        # Two controlled gates
        gate1 = g.add_vertex(zx.VertexType.X, qubit=0, row=2, phase=Fraction(1))
        gate2 = g.add_vertex(zx.VertexType.Z, qubit=1, row=2, phase=Fraction(1))

        # Connect control to both gates
        g.add_edge((c_in, control))
        g.add_edge((control, gate1))
        g.add_edge((control, gate2))

        # Connect quantum flow
        g.add_edge((q1, gate1))
        g.add_edge((q2, gate2))

        # Outputs
        out1 = g.add_vertex(zx.VertexType.BOUNDARY, qubit=0, row=3)
        out2 = g.add_vertex(zx.VertexType.BOUNDARY, qubit=1, row=3)
        g.add_edge((gate1, out1))
        g.add_edge((gate2, out2))

        g.set_inputs((q1, q2, c_in))
        g.set_outputs((out1, out2))

        print(f"Fanout control circuit created. Grounds: {len(g.grounds())}, Hybrid: {g.is_hybrid()}\n")

    def adaptive_measurement_cascade(self):
        """Models a dynamic circuit with cascading measurements."""
        print("9. Adaptive Measurement Cascade")
        print("-" * 30)

        g = zx.Graph()

        q = g.add_vertex(zx.VertexType.BOUNDARY, qubit=0, row=0)

        # First measurement and conditional gate
        meas1 = g.add_vertex(zx.VertexType.Z, qubit=0, row=1, ground=True)
        ctrl1 = g.add_vertex(zx.VertexType.Z, qubit=1, row=2, ground=True)
        gate1 = g.add_vertex(zx.VertexType.X, qubit=0, row=3, phase=Fraction(1))

        # Second measurement and conditional gate
        meas2 = g.add_vertex(zx.VertexType.Z, qubit=0, row=4, ground=True)
        ctrl2 = g.add_vertex(zx.VertexType.Z, qubit=2, row=5, ground=True)
        gate2 = g.add_vertex(zx.VertexType.Z, qubit=0, row=6, phase=Fraction(1))

        out = g.add_vertex(zx.VertexType.BOUNDARY, qubit=0, row=7)

        g.add_edge((q, meas1))
        g.add_edge((meas1, ctrl1))
        g.add_edge((ctrl1, gate1))
        g.add_edge((gate1, meas2))
        g.add_edge((meas2, ctrl2))
        g.add_edge((ctrl2, gate2))
        g.add_edge((gate2, out))

        g.set_inputs((q,))
        g.set_outputs((out,))

        print(f"Cascading feedback circuit created. Grounds: {len(g.grounds())}, Is hybrid: {g.is_hybrid()}\n")

if __name__ == "__main__":
    demo = ExtendedHybridCircuitDemo()
    demo.classical_feedback_loop()
    demo.classical_fanout_and_condition()
    demo.adaptive_measurement_cascade()

=== Extended PyZX Hybrid Quantum-Classical Circuit Demo ===

7. Classical Feedback Loop
------------------------------
Vertices: 6, Grounds: 3, Is hybrid: True

8. Classical Fan-Out and Conditional Gates
------------------------------
Fanout control circuit created. Grounds: 2, Hybrid: True

9. Adaptive Measurement Cascade
------------------------------
Cascading feedback circuit created. Grounds: 4, Is hybrid: True



In [63]:
import pyzx as zx
from fractions import Fraction

class ExtendedHybridCircuitDemo:
    def __init__(self):
        print("=== Extended PyZX Hybrid Quantum-Classical Circuit Demo ===\n")

    def classical_feedback_loop(self):
        """7. Classical Feedback Loop: Demonstrates classical feedback control with a loop."""
        print("7. Classical Feedback Loop")
        print("-" * 30)

        g = zx.Graph()

        # Quantum input
        q_in = g.add_vertex(zx.VertexType.BOUNDARY, qubit=0, row=0)

        # Initial measurement
        meas = g.add_vertex(zx.VertexType.Z, qubit=0, row=1, ground=True)

        # Classical processing and feedback gate
        classical_proc = g.add_vertex(zx.VertexType.Z, qubit=1, row=2, ground=True)
        feedback_gate = g.add_vertex(zx.VertexType.X, qubit=0, row=3, phase=Fraction(1))

        # Final measurement
        final_meas = g.add_vertex(zx.VertexType.Z, qubit=0, row=4, ground=True)
        q_out = g.add_vertex(zx.VertexType.BOUNDARY, qubit=0, row=5)

        # Connect components
        g.add_edge((q_in, meas))
        g.add_edge((meas, classical_proc))
        g.add_edge((classical_proc, feedback_gate))
        g.add_edge((feedback_gate, final_meas))
        g.add_edge((final_meas, q_out))

        g.set_inputs((q_in,))
        g.set_outputs((q_out,))

        print(f"Vertices: {g.num_vertices()}, Grounds: {len(g.grounds())}, Is hybrid: {g.is_hybrid()}\n")

    def classical_fanout_and_condition(self):
        """8. Classical Fan-Out and Conditional Gates: multiple qubits controlled by one classical bit."""
        print("8. Classical Fan-Out and Conditional Gates")
        print("-" * 30)

        g = zx.Graph()

        # Classical control input
        c_in = g.add_vertex(zx.VertexType.BOUNDARY, qubit=2, row=0, ground=True)
        control = g.add_vertex(zx.VertexType.Z, qubit=2, row=1, ground=True)

        # Two quantum inputs
        q1 = g.add_vertex(zx.VertexType.BOUNDARY, qubit=0, row=0)
        q2 = g.add_vertex(zx.VertexType.BOUNDARY, qubit=1, row=0)

        # Two controlled gates
        gate1 = g.add_vertex(zx.VertexType.X, qubit=0, row=2, phase=Fraction(1))
        gate2 = g.add_vertex(zx.VertexType.Z, qubit=1, row=2, phase=Fraction(1))

        # Connect control to both gates
        g.add_edge((c_in, control))
        g.add_edge((control, gate1))
        g.add_edge((control, gate2))

        # Connect quantum flow
        g.add_edge((q1, gate1))
        g.add_edge((q2, gate2))

        # Outputs
        out1 = g.add_vertex(zx.VertexType.BOUNDARY, qubit=0, row=3)
        out2 = g.add_vertex(zx.VertexType.BOUNDARY, qubit=1, row=3)
        g.add_edge((gate1, out1))
        g.add_edge((gate2, out2))

        g.set_inputs((q1, q2, c_in))
        g.set_outputs((out1, out2))

        print(f"Fanout control circuit created. Grounds: {len(g.grounds())}, Hybrid: {g.is_hybrid()}\n")

    def adaptive_measurement_cascade(self):
        """9. Adaptive Measurement Cascade: dynamic circuit with cascading measurements."""
        print("9. Adaptive Measurement Cascade")
        print("-" * 30)

        g = zx.Graph()

        q = g.add_vertex(zx.VertexType.BOUNDARY, qubit=0, row=0)

        # First measurement and conditional gate
        meas1 = g.add_vertex(zx.VertexType.Z, qubit=0, row=1, ground=True)
        ctrl1 = g.add_vertex(zx.VertexType.Z, qubit=1, row=2, ground=True)
        gate1 = g.add_vertex(zx.VertexType.X, qubit=0, row=3, phase=Fraction(1))

        # Second measurement and conditional gate
        meas2 = g.add_vertex(zx.VertexType.Z, qubit=0, row=4, ground=True)
        ctrl2 = g.add_vertex(zx.VertexType.Z, qubit=2, row=5, ground=True)
        gate2 = g.add_vertex(zx.VertexType.Z, qubit=0, row=6, phase=Fraction(1))

        out = g.add_vertex(zx.VertexType.BOUNDARY, qubit=0, row=7)

        g.add_edge((q, meas1))
        g.add_edge((meas1, ctrl1))
        g.add_edge((ctrl1, gate1))
        g.add_edge((gate1, meas2))
        g.add_edge((meas2, ctrl2))
        g.add_edge((ctrl2, gate2))
        g.add_edge((gate2, out))

        g.set_inputs((q,))
        g.set_outputs((out,))

        print(f"Cascading feedback circuit created. Grounds: {len(g.grounds())}, Is hybrid: {g.is_hybrid()}\n")

if __name__ == "__main__":
    demo = ExtendedHybridCircuitDemo()
    demo.classical_feedback_loop()
    demo.classical_fanout_and_condition()
    demo.adaptive_measurement_cascade()


=== Extended PyZX Hybrid Quantum-Classical Circuit Demo ===

7. Classical Feedback Loop
------------------------------
Vertices: 6, Grounds: 3, Is hybrid: True

8. Classical Fan-Out and Conditional Gates
------------------------------
Fanout control circuit created. Grounds: 2, Hybrid: True

9. Adaptive Measurement Cascade
------------------------------
Cascading feedback circuit created. Grounds: 4, Is hybrid: True




## PyZX Hybrid Quantum-Classical Circuit Extension
================================================

This extension builds on the initial PyZX hybrid circuit demo, exploring more
advanced hybrid functionality such as classical-quantum feedback loops,
classical fan-out, and conditional resets, while preserving the code's
structure and styling.


## 1. Classical Fan-Out Control
Goal: A single classical bit controls multiple quantum gates simultaneously.

Implementation details:

The classical input bit is created as a grounded boundary vertex (ground=True).

This bit fans out through a grounded Z-spider vertex to multiple quantum-controlled X gates acting on separate qubits.

Quantum inputs are connected to these X gates, which output to boundary vertices.

Significance: This simulates classical control logic broadcasting to different quantum subsystems.

Key Points:

Use of ground=True to mark classical control vertices.

Multiple quantum gates conditioned on the same classical signal.

Inputs include both classical control and quantum registers; outputs are quantum states after conditional gates.

## 2. Conditional Qubit Reset
Goal: Implement a quantum reset operation conditioned on the result of a mid-circuit measurement.

Implementation details:

A quantum input is measured using a grounded Z-spider vertex.

The measurement result feeds a classical processing vertex (also grounded).

This classical signal controls a Pauli-X gate acting as a reset operation.

The final quantum state is output through a boundary vertex.

Significance: Demonstrates mid-circuit measurement feedback commonly required in error correction or state preparation protocols.

Key Points:

Grounded vertices track classical info flowing from measurement to reset logic.

The X gate acts as a conditional bit flip (reset) controlled by classical data.

## 3. Multi-Classical Feedback Paths
Goal: Represent multiple measurement outcomes feeding into distinct classical processing units.

Implementation details:

Two quantum inputs are measured individually, creating two classical bits (grounded Z vertices).

Each classical bit feeds into separate classical processors (grounded Z vertices).

Processed classical signals lead to classical output boundary vertices (grounded).




In [64]:
import pyzx as zx
from fractions import Fraction
import numpy as np
from typing import List, Tuple, Dict, Set

class HybridCircuitExtension:
    """Extended demonstrations of hybrid circuits using PyZX."""

    def __init__(self):
        print("=== Extended Hybrid Quantum-Classical Circuit Tests ===\n")

    def classical_fanout(self):
        """Classical bit controlling multiple quantum gates."""
        print("7. Classical Fan-Out Control")
        print("-" * 30)

        g = zx.Graph()

        # Classical control input (grounded)
        c_input = g.add_vertex(zx.VertexType.BOUNDARY, qubit=0, row=0, ground=True)

        # Control fanout vertex
        fanout = g.add_vertex(zx.VertexType.Z, qubit=0, row=1, ground=True)

        # Quantum inputs and controlled X gates
        q_gates = []
        q_outputs = []
        for i in range(3):
            q_in = g.add_vertex(zx.VertexType.BOUNDARY, qubit=i+1, row=0)
            x_gate = g.add_vertex(zx.VertexType.X, qubit=i+1, row=2, phase=Fraction(1))
            q_out = g.add_vertex(zx.VertexType.BOUNDARY, qubit=i+1, row=3)

            g.add_edge((q_in, x_gate))
            g.add_edge((x_gate, q_out))
            g.add_edge((fanout, x_gate))

            q_gates.append(x_gate)
            q_outputs.append(q_out)

        g.add_edge((c_input, fanout))

        g.set_inputs((c_input,) + tuple(range(1, 4)))
        g.set_outputs(tuple(q_outputs))

        print("Classical fan-out created")
        print(f"Total vertices: {g.num_vertices()}")
        print(f"Classical vertices (grounded): {len(g.grounds())}")
        print(f"Is hybrid: {g.is_hybrid()}")
        print()

    def conditional_reset(self):
        """Conditional reset of qubit based on classical feedback."""
        print("8. Conditional Qubit Reset")
        print("-" * 30)

        g = zx.Graph()

        # Quantum input
        q_in = g.add_vertex(zx.VertexType.BOUNDARY, qubit=0, row=0)

        # Measurement
        measure = g.add_vertex(zx.VertexType.Z, qubit=0, row=1, ground=True)

        # Classical processing of measurement
        feedback = g.add_vertex(zx.VertexType.Z, qubit=1, row=2, ground=True)

        # Conditional reset (e.g., apply X gate if measurement = 1)
        x_reset = g.add_vertex(zx.VertexType.X, qubit=0, row=3, phase=Fraction(1))

        # Output
        q_out = g.add_vertex(zx.VertexType.BOUNDARY, qubit=0, row=4)

        # Connections
        g.add_edge((q_in, measure))
        g.add_edge((measure, feedback))
        g.add_edge((feedback, x_reset))
        g.add_edge((x_reset, q_out))

        g.set_inputs((q_in,))
        g.set_outputs((q_out,))

        print("Conditional reset circuit created")
        print(f"Vertices: {g.num_vertices()}")
        print(f"Ground vertices: {g.grounds()}")
        print(f"Is hybrid: {g.is_hybrid()}")
        print()

    def multi_classical_path(self):
        """Multiple classical paths from measurements to separate processing units."""
        print("9. Multi-Classical Feedback Paths")
        print("-" * 30)

        g = zx.Graph()

        # Quantum register
        q0 = g.add_vertex(zx.VertexType.BOUNDARY, qubit=0, row=0)
        q1 = g.add_vertex(zx.VertexType.BOUNDARY, qubit=1, row=0)

        # Measurements
        m0 = g.add_vertex(zx.VertexType.Z, qubit=0, row=1, ground=True)
        m1 = g.add_vertex(zx.VertexType.Z, qubit=1, row=1, ground=True)

        # Classical processing units
        proc0 = g.add_vertex(zx.VertexType.Z, qubit=2, row=2, ground=True)
        proc1 = g.add_vertex(zx.VertexType.Z, qubit=3, row=2, ground=True)

        # Output boundaries
        out0 = g.add_vertex(zx.VertexType.BOUNDARY, qubit=2, row=3, ground=True)
        out1 = g.add_vertex(zx.VertexType.BOUNDARY, qubit=3, row=3, ground=True)

        # Connections
        g.add_edge((q0, m0))
        g.add_edge((q1, m1))
        g.add_edge((m0, proc0))
        g.add_edge((m1, proc1))
        g.add_edge((proc0, out0))
        g.add_edge((proc1, out1))

        g.set_inputs((q0, q1))
        g.set_outputs((out0, out1))

        print("Multiple classical paths created")
        print(f"Vertices: {g.num_vertices()}")
        print(f"Ground vertices: {g.grounds()}")
        print(f"Is hybrid: {g.is_hybrid()}")
        print()


if __name__ == "__main__":
    demo = HybridCircuitExtension()
    demo.classical_fanout()
    demo.conditional_reset()
    demo.multi_classical_path()


=== Extended Hybrid Quantum-Classical Circuit Tests ===

7. Classical Fan-Out Control
------------------------------
Classical fan-out created
Total vertices: 11
Classical vertices (grounded): 2
Is hybrid: True

8. Conditional Qubit Reset
------------------------------
Conditional reset circuit created
Vertices: 5
Ground vertices: {1, 2}
Is hybrid: True

9. Multi-Classical Feedback Paths
------------------------------
Multiple classical paths created
Vertices: 8
Ground vertices: {2, 3, 4, 5, 6, 7}
Is hybrid: True

